In [65]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, homogeneity_completeness_v_measure
from keras.layers import Input, Embedding, LSTM, Dense, Dropout
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping

In [2]:
# load the data
file_name = 'names.csv'
data = pd.read_csv(file_name, header=0, sep=',')

# remove English for easy prediction
data = data[data['nation'] != 'ENGLISH']
data

,first_name,last_name,nation
0,PONGSIRI,SUASAT,THAI
1,NOPADON,MISANG,THAI
2,KRISADA,SOMSUK,THAI
3,WORAWUT,SANGANGAM,THAI
4,KULPRIYA,KITCHAROEN,THAI
5,PATTHARAPONG,PIMONSAKONWONG,THAI
6,KANITTA,SUWANNARAT,THAI
7,JEDSADAKORN,PANICH,THAI
8,THITAPORN,KONKONG,THAI
9,DARUNWAN,THONGKHOENKHUN,THAI


In [3]:
# convert name to lower case
data['first_name'] = data['first_name'].str.lower()
data['last_name'] = data['last_name'].str.lower()
# data['full_name'] = data['first_name'] + ' ' + data['last_name']
data.head()

,first_name,last_name,nation
0,pongsiri,suasat,THAI
1,nopadon,misang,THAI
2,krisada,somsuk,THAI
3,worawut,sangangam,THAI
4,kulpriya,kitcharoen,THAI


# Analyze name length

In [4]:
data['first_name_len'] = data['first_name'].str.len()
data['last_name_len'] = data['last_name'].str.len()
# data['full_name_len'] = data['full_name'].str.len()
data.head()

,first_name,last_name,nation,first_name_len,last_name_len
0,pongsiri,suasat,THAI,8,6
1,nopadon,misang,THAI,8,6
2,krisada,somsuk,THAI,7,6
3,worawut,sangangam,THAI,7,9
4,kulpriya,kitcharoen,THAI,8,10


In [5]:
data.describe()

,first_name_len,last_name_len
count,300.000000,300.000000
mean,5.940000,6.580000
std,2.418518,2.788582
min,2.000000,2.000000
25%,4.000000,5.000000
50%,6.000000,6.000000
75%,8.000000,8.000000
max,12.000000,16.000000


# Preprocessing name for machine learning model
We need to convert name alphabets into a fixed length matrix, we also need to specify the maximum name
length. E.g. if the maximum name length is 10, and the English alphabets are 26. Then the final vector length
is 10*26 which is 260 for each name.

In [6]:
max_name_len = data['first_name_len'].max()
max_name_len = max(data['last_name_len'].max(), max_name_len)
# max_name_len = max(data['full_name_len'].max(), max_name_len)
max_name_len

16

In [7]:
import string
allowed_chars = string.ascii_lowercase + '- '
allowed_chars, len(allowed_chars)

('abcdefghijklmnopqrstuvwxyz- ', 28)

In [8]:
# convert name string into (name_len, len(allowed_chars)) shaped matrix containing one-hot values
# if the character is present
def name_to_matrix(name, max_name_len=None):
    vectors = []
    vec_shape = [1, len(allowed_chars)]
    for c in name:
        idx = None # contains index a=0, b=1, c=2, ..., z=25
        vec = np.zeros(vec_shape, dtype=np.float32)
        if c in allowed_chars:
            idx = allowed_chars.index(c)
            vec[0,idx] = 1.0
        vectors.append(vec)
    if max_name_len is not None:
        n_blank_chars = max_name_len - len(name)
        for i in range(n_blank_chars):
            blank_vec = np.zeros(vec_shape, dtype=np.float32)
            vectors.append(blank_vec)
    return np.concatenate(vectors, axis=0)

name_matrix = name_to_matrix('abz', max_name_len)
name_matrix[:5, :], name_matrix.shape

(array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.]], dtype=float32), (16, 28))

# Aggregating the dataset
Use the function defined to aggregate the dataset

In [9]:
available_nations = np.array(list(set(data['nation'])))
available_nations

array(['CHINESE', 'THAI', 'JAPANESE'], 
      dtype='<U8')

In [10]:
def one_hot(idx, size):
    vec = np.zeros([size], dtype=np.float32)
    vec[idx] = 1.0
    return vec

one_hot(1, 5)

array([ 0.,  1.,  0.,  0.,  0.], dtype=float32)

In [11]:
# apply name_to_matrix() function to all the names and aggregate the results
X, y, X_str, y_nation, y_pos = [], [], [], [], []
name_cols = ['first_name', 'last_name']
# name_cols = ['full_name']
for name_col_idx, name_col in enumerate(name_cols):
    for idx, row in data.iterrows():
        name_matrix = name_to_matrix(row[name_col], max_name_len)
        X.append(name_matrix[np.newaxis])
        y.append([row['nation'], name_col])
        X_str.append(row[name_col])
        y_nation.append(one_hot(available_nations.tolist().index(row['nation']), len(available_nations))[np.newaxis])
        y_pos.append(one_hot(name_col_idx, len(name_cols))[np.newaxis])
X = np.concatenate(X, axis=0)
y = np.array(y)
X_str = np.array(X_str)
y_nation = np.concatenate(y_nation, axis=0)
y_pos = np.concatenate(y_pos, axis=0)
X.shape, y.shape, y_nation.shape, y_pos.shape

((600, 16, 28), (600, 2), (600, 3), (600, 2))

In [12]:
X_train, X_test, y_train, y_test, X_str_train, X_str_test,\
    y_nation_train, y_nation_test, y_pos_train, y_pos_test = \
    train_test_split(X, y, X_str, y_nation, y_pos, test_size=0.2)
X_train.shape, X_test.shape, y_nation_train.shape, y_nation_test.shape

((480, 16, 28), (120, 16, 28), (480, 3), (120, 3))

In [13]:
def flatten_X(X):
    return X.reshape([X.shape[0], -1])
X_train_flat = flatten_X(X_train)
X_test_flat = flatten_X(X_test)
X_train_flat.shape, X_test_flat.shape

((480, 448), (120, 448))

# Train a machine learning model
Use a simple model, or a deep one.

In [ ]:
# # train a simple model and show the score
# model_names = ['nation', 'first/last']
# models = dict()
# for i in range(y.shape[1]):
#     simple_model = DecisionTreeClassifier()
#     simple_model.fit(X_train_flat, y_train[:,i])
#     print(model_names[i], simple_model.score(X_train_flat, y_train[:,i]), simple_model.score(X_test_flat, y_test[:,i]))
#     models[model_names[i]] = simple_model

In [14]:
# define a deep model
name_input = Input(shape=(X_train_flat.shape[1],), name='name_input', dtype='float32')
hidden = Dense(64, activation='relu')(name_input)
hidden = Dropout(0.5)(hidden)
hidden = Dense(64, activation='relu')(hidden)
hidden = Dropout(0.5)(hidden)
hidden = Dense(64, activation='relu')(hidden)
hidden = Dropout(0.5)(hidden)
out_nation = Dense(y_nation.shape[1], activation='softmax', name='out_nation')(hidden)
out_pos = Dense(y_pos.shape[1], activation='softmax', name='out_pos')(hidden)

model = Model(inputs=[name_input], outputs=[out_nation, out_pos])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
name_input (InputLayer)          (None, 448)           0                                            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 64)            28736                                        
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 64)            0                                            
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 64)            4160                                         
___________________________________________________________________________________________

In [15]:
# train it
callbacks = [EarlyStopping(patience=10)]
model.fit([X_train_flat], [y_nation_train, y_pos_train], epochs=100, validation_split=0.1, callbacks=callbacks, batch_size=32)

Train on 432 samples, validate on 48 samples
Epoch 1/100
432/432 [==============================] - 1s - loss: 1.8158 - out_nation_loss: 1.1139 - out_pos_loss: 0.7019 - out_nation_acc: 0.3241 - out_pos_acc: 0.4884 - val_loss: 1.7941 - val_out_nation_loss: 1.0949 - val_out_pos_loss: 0.6992 - val_out_nation_acc: 0.3542 - val_out_pos_acc: 0.4375
Epoch 2/100
432/432 [==============================] - 0s - loss: 1.8133 - out_nation_loss: 1.1135 - out_pos_loss: 0.6999 - out_nation_acc: 0.3310 - out_pos_acc: 0.4769 - val_loss: 1.7937 - val_out_nation_loss: 1.0924 - val_out_pos_loss: 0.7013 - val_out_nation_acc: 0.3958 - val_out_pos_acc: 0.5208
Epoch 3/100
432/432 [==============================] - 0s - loss: 1.7980 - out_nation_loss: 1.0949 - out_pos_loss: 0.7031 - out_nation_acc: 0.3704 - out_pos_acc: 0.5023 - val_loss: 1.7887 - val_out_nation_loss: 1.0876 - val_out_pos_loss: 0.7011 - val_out_nation_acc: 0.4167 - val_out_pos_acc: 0.4583
Epoch 4/100
432/432 [==============================] - 

In [16]:
# performance evaluation
df = pd.DataFrame(columns=['loss','out_nation_loss','out_pos_loss','out_nation_acc', 'out_pos_acc'], index=['train', 'test'])
df.loc['train'] = model.evaluate([X_train_flat], [y_nation_train, y_pos_train], verbose=0)
df.loc['test'] = model.evaluate([X_test_flat], [y_nation_test, y_pos_test], verbose=0)
df

,loss,out_nation_loss,out_pos_loss,out_nation_acc,out_pos_acc
train,0.474083,0.0846203,0.389463,0.98125,0.847917
test,1.04598,0.328438,0.71754,0.9,0.691667


# Clustering
Try clustering to see if its accuracy would be better or worse than neural network.

We will map the most common cluster to the most common label and so on.
And evaluate accuracy from the induced label.

Note that we will only use the nation label and not the first/last name indicator label.

In [83]:
%%time
kmeans = KMeans(n_clusters=y_nation_train.shape[1])
km_pred_train = kmeans.fit_predict(X_train_flat) # predicted clusters
km_pred_test = kmeans.predict(X_test_flat)

Wall time: 212 ms


In [84]:
cluster_freq = Counter(km_pred_train).most_common()
cluster_freq

[(0, 236), (2, 123), (1, 121)]

In [85]:
label_freq = Counter(y_nation_train.argmax(axis=1)).most_common()
label_freq

[(0, 166), (1, 158), (2, 156)]

In [86]:
cluster2label = np.empty([len(cluster_freq)], dtype=np.int32)
for cluster, label in zip(cluster_freq, label_freq):
    cluster = cluster[0]
    label = label[0]
    cluster2label[cluster] = label
cluster2label

array([0, 2, 1])

In [87]:
km_pred_train_label = cluster2label[km_pred_train]
km_pred_test_label = cluster2label[km_pred_test]
score_train = accuracy_score(y_nation_train.argmax(axis=1), km_pred_train_label)
score_test = accuracy_score(y_nation_test.argmax(axis=1), km_pred_test_label)
score_train, score_test

(0.34791666666666665, 0.38333333333333336)

In [88]:
homogeneity_completeness_v_measure(y_nation_train.argmax(axis=1), km_pred_train)

(0.011250360989112387, 0.011819783284475619, 0.011528044834610462)

In [95]:
homogeneity_completeness_v_measure(y_nation_test.argmax(axis=1), km_pred_test)

(0.021533528435585605, 0.022537385064215959, 0.022024023720144663)

In [103]:
# clustering results
table = {
    'Name': X_str_test,
    'Prediction': available_nations[km_pred_test_label],
    'True Label': available_nations[y_nation_test.argmax(axis=1)],
    'Accurate?': (km_pred_test_label == y_nation_test.argmax(axis=1))
}
pd.DataFrame(data=table).sort_values('Accurate?', ascending=False)

,Accurate?,Name,Prediction,True Label
27,True,jingyi,CHINESE,CHINESE
73,True,haruyama,JAPANESE,JAPANESE
54,True,gu,CHINESE,CHINESE
66,True,chanwiset,THAI,THAI
65,True,khualamthan,THAI,THAI
30,True,hanazawa,JAPANESE,JAPANESE
77,True,ruogang,CHINESE,CHINESE
99,True,thanakorn,THAI,THAI
98,True,ling,CHINESE,CHINESE
34,True,matsuo,JAPANESE,JAPANESE


# Save/Load model

In [17]:
model.save('model.h5')

In [18]:
model = load_model('model.h5')

# Inspection
Identify names that the model predicts incorrectly

In [19]:
incorrect_mask = model.predict(X_test_flat)[0].argmax(axis=1) != y_nation_test.argmax(axis=1)
names = X_str_test[incorrect_mask]
pred = model.predict(X_test_flat[incorrect_mask,:])[0]
pred_nations = available_nations[pred.argmax(axis=1)]
probs = pred.max(axis=1)
nations = available_nations[y_nation_test[incorrect_mask].argmax(axis=1)]
df = pd.DataFrame(columns=['name', 'predicted nation', 'true nation', 'prediction confidence'])
for i in range(len(names)):
    df.loc[i] = [names[i], pred_nations[i], nations[i], probs[i]*100]
df.sort_values('prediction confidence', ascending=False)

,name,predicted nation,true nation,prediction confidence
6,somsuk,JAPANESE,THAI,99.862552
8,kanbayashi,THAI,JAPANESE,99.774545
2,kin,CHINESE,JAPANESE,98.863739
11,muranaka,THAI,JAPANESE,97.508121
10,yeanjai,CHINESE,THAI,89.919162
4,sosuepngam,JAPANESE,THAI,85.743368
3,botan,THAI,JAPANESE,84.878314
5,ruogang,THAI,CHINESE,77.324808
0,koichi,THAI,JAPANESE,62.582999
9,artit,JAPANESE,THAI,57.349581


In [20]:
# count nations that are incorrectly predicted by the model
from collections import Counter
Counter(df['true nation'])

Counter({'CHINESE': 1, 'JAPANESE': 6, 'THAI': 5})

In [21]:
# check if a certain name is inside a dataset
X_that_we_care = X_str_train
X_that_we_care[np.array(['kr' in x for x in X_that_we_care])]

array(['kraingam', 'krittanan', 'krisada'], 
      dtype='<U16')

# Inference
Given a name, predict the nation.

In [22]:
def predict(names, model, max_name_len):
    assert(type(names) is list)
    X = []
    for name in names:
        matrix = name_to_matrix(name.lower(), max_name_len)[np.newaxis]
        X.append(matrix)
    X = np.concatenate(X, axis=0)
    X_flat = flatten_X(X)
    return model.predict(X_flat)

names = ["Zemin", "Sunisa", "Phakphum", "Krittitam", "Juksukit", "Chanchana", "Sornsoontorn", "Xi",
        "Jinping", "Shinzo", "Abe", "Prayut", "Chan-o-char", "Watcharaphong",
        "Pusit", "Sora"]
names = [name.lower() for name in names]
pred = predict(names, model, max_name_len)
pred_nation, pred_pos = pred
df = pd.DataFrame(columns=['name', 'nation'] + available_nations.tolist() + ['position'] + name_cols + ['In Training Set?'])
for i in range(len(names)):
    best_nation = available_nations[pred_nation[i,:].argmax()]
    best_pos = name_cols[pred_pos[i,:].argmax()]
    in_training = names[i] in X_str_train
    df.loc[i] = [names[i], best_nation] + (pred_nation[i,:]*100).tolist() + [best_pos] + (pred_pos[i,:]*100).tolist() + [in_training]
df

,name,nation,CHINESE,THAI,JAPANESE,position,first_name,last_name,In Training Set?
0,zemin,CHINESE,98.161499,0.430662,1.407845,last_name,22.704750,77.295258,True
1,sunisa,THAI,4.621304,66.424988,28.953711,first_name,80.570312,19.429689,False
2,phakphum,THAI,0.030125,99.768600,0.201267,last_name,2.777356,97.222641,False
3,krittitam,THAI,0.008759,96.771255,3.220000,first_name,91.955147,8.044858,False
4,juksukit,JAPANESE,0.124522,3.535109,96.340363,last_name,13.979593,86.020409,False
5,chanchana,THAI,0.032551,99.542007,0.425449,first_name,73.261993,26.738012,True
6,sornsoontorn,THAI,0.000013,99.997421,0.002565,last_name,1.524175,98.475822,True
7,xi,CHINESE,99.783173,0.079525,0.137299,first_name,53.744759,46.255241,False
8,jinping,CHINESE,57.653511,41.831242,0.515245,last_name,9.156750,90.843246,False
9,shinzo,JAPANESE,6.478907,2.396963,91.124138,last_name,15.049924,84.950081,False
